<a href="https://colab.research.google.com/github/fsaudm/Streaming_QLlama3.1/blob/main/Streaming_QLlama3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streaming Text Generation with a 4-bit Quantized Llama 3.1 model

In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U bitsandbytes
# !pip install -q python-dotenv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.0 MB/s eta 0:00:00


In [2]:
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import torch
import warnings

## To import secrets from Colab Secrets
from google.colab import userdata

In [3]:
# Set environment variables
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

# Set model
model_id = "fsaudm/Meta-Llama-3.1-8B-Instruct-NF4"


# Load tokenizer
load_tokenizer_start = time.time()
tokenizer = AutoTokenizer.from_pretrained(model_id)
load_tokenizer_end = time.time()
print(f"Time to load tokenizer: {load_tokenizer_end - load_tokenizer_start:.2f} seconds")


# Load model
load_model_start = time.time()
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    device_map="auto", ### Loads to multiple devices!!
    torch_dtype=torch.bfloat16,
    )
load_model_end = time.time()
print(f"Time to load model: {load_model_end - load_model_start:.2f} seconds")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Time to load tokenizer: 3.87 seconds


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Time to load model: 221.83 seconds


In [4]:
def generate_response(user_input, model, tokenizer, max_length=1000):
    """
    Generates a response using a text generation model with streaming output.

    Parameters:
    - user_input: A string containing the user's input.
    - model_id: The name of the model to use for text generation.
    - max_length: The maximum length of the generated text.

    Returns:
    - The generated response from the model.
    """

    # Streamer
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    # Model input (tokenized text)
    inputs = tokenizer(user_input, return_tensors="pt").to('cuda')

    start_time = time.time()
    generation = model.generate(**inputs, streamer=streamer, max_new_tokens=max_length)
    end_time = time.time()

    time_taken = end_time - start_time
    print(f"\nTime taken to generate: {time_taken:.2f} seconds")

    return

In [6]:
# Example usage
prompt = "How to become a data scientist?"
generate_response(prompt, model, tokenizer, max_length=50)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 A guide to getting started
How to become a data scientist? A guide to getting started
Data science is a field that combines computer science, statistics, and domain-specific knowledge to extract insights from data. As a data scientist, you will work with

Time taken to generate: 7.25 seconds
